In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)
    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[ 0.44149232, -0.544083  ]]], dtype=float32)


In [4]:
with tf.variable_scope('3_batches') as scope:
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
    outpus, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
    

array([[[-0.41866544,  0.47120455]]], dtype=float32)


In [23]:
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
# x_one_hot = tf.one_hot(x_data,depth=5, dtype=tf.int32)
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
# pp.pprint(x_one_hot.eval())
# pp.pprint(type(x_one_hot))
y_data = [[1, 0, 2, 3, 3, 4]]    # ihello
idx2char = ['h', 'i', 'e', 'l', 'o']

In [6]:
num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

In [7]:
# Placeholder
X = tf.placeholder(tf.float32, shape=[None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])

In [8]:
# Cell
cell = rnn.BasicLSTMCell(hidden_size)
initial_state = cell.zero_state(batch_size, dtype=tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

In [26]:
outputs.shape

TensorShape([Dimension(1), Dimension(6), Dimension(5)])

In [9]:
#Fully connected layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)
outputs = tf.reshape(outputs, shape=[batch_size, sequence_length, num_classes])

In [10]:
#loss, Optimizer
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights) #weight? 의미가 뭐지?
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [30]:
test = tf.reshape([1,2,3],[3])
print(test.eval())

[1 2 3]


In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        sess.run(train, feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

	Prediction str:  llllll
	Prediction str:  llllll
	Prediction str:  llllll
	Prediction str:  lllllo
	Prediction str:  lllllo
	Prediction str:  lllllo
	Prediction str:  leello
	Prediction str:  leello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
	Prediction str:  ihello
